In [1]:
import Pkg; Pkg.add("Tables")
using CSV, Tables
using JuMP
using Gurobi

import Pkg; Pkg.add("JSON3")
using JSON3

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-04


#### Load in Data

In [3]:
all_classes = open("data/all_classes_info.json", "r") do file 
    JSON3.read(file)
end

R = CSV.File("data/ratings.csv",header=0) |> Tables.matrix;
H = CSV.File("data/hours.csv",header=0) |> Tables.matrix;
U = CSV.File("data/units.csv",header=0) |> Tables.matrix;

vars = CSV.File("data/variables.csv",header=0) |> Tables.matrix;
n = vars[1];
S = vars[2];

In [4]:
random_class = all_classes[rand(1:length(all_classes))]

JSON3.Object{Base.CodeUnits{UInt8, String}, SubArray{UInt64, 1, Vector{UInt64}, Tuple{UnitRange{Int64}}, true}} with 24 entries:
  :rating             => 6.6
  :has_final          => false
  :description        => "Facilitates design and construction of installations …
  :offered_fall       => false
  :offered_spring     => false
  :out_of_class_hours => 6.83
  :total_units        => 6
  :related_subjects   => ["17.391", "17.506", "17.830", "17.803", "17.565", "17…
  :pdf_option         => false
  :in_class_hours     => 9.47
  :is_half_class      => false
  :level              => "U"
  :url                => "http://student.mit.edu/catalog/m17b.html#17.922"
  :subject_id         => "17.922"
  :title              => "Martin Luther King, Jr. Design Seminar"
  :lab_units          => 0
  :design_units       => 0
  :public             => true
  :offered_summer     => false
  :lecture_units      => 3
  :preparation_units  => 3
  :enrollment_number  => 30
  :is_variable_units  => false
  :off

In [5]:
subjectid_to_index = Dict(class[:subject_id] => ind for (ind, class) in enumerate(all_classes))
index_to_subjectid = Dict(ind => class[:subject_id] for (ind, class) in enumerate(all_classes))

Dict{Int64, String} with 4704 entries:
  4700 => "WGS.301"
  4576 => "STS.022"
  2288 => "21G.905"
  1703 => "18.C20"
  1956 => "20.101"
  2350 => "21H.226"
  3406 => "6.2060"
  2841 => "24.234"
  2876 => "24.906"
  687  => "12.751"
  185  => "1.THG"
  1090 => "15.518"
  2015 => "20.560"
  3293 => "5.00"
  1704 => "18.S096"
  3220 => "4.608"
  422  => "11.353"
  1266 => "16.004"
  183  => "1.S992"
  1823 => "2.670"
  4030 => "9.77"
  551  => "12.005"
  3324 => "5.39"
  2119 => "21G.056"
  3298 => "5.062"
  ⋮    => ⋮

#### Defining the Basic Optimization Problem

In [6]:
function create_model(env) 
    return  Model(() -> Gurobi.Optimizer(env))
end
model1 = create_model(GRB_ENV)

A JuMP Model
├ solver: Gurobi
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [7]:
function add_course_semester_variables!(model::JuMP.Model)
    @variable(model, [1:n, 1:S], Bin)
end
model_var = add_course_semester_variables!(model1)

4704×8 Matrix{VariableRef}:
 _[1]     _[4705]  _[9409]   _[14113]  _[18817]  _[23521]  _[28225]  _[32929]
 _[2]     _[4706]  _[9410]   _[14114]  _[18818]  _[23522]  _[28226]  _[32930]
 _[3]     _[4707]  _[9411]   _[14115]  _[18819]  _[23523]  _[28227]  _[32931]
 _[4]     _[4708]  _[9412]   _[14116]  _[18820]  _[23524]  _[28228]  _[32932]
 _[5]     _[4709]  _[9413]   _[14117]  _[18821]  _[23525]  _[28229]  _[32933]
 _[6]     _[4710]  _[9414]   _[14118]  _[18822]  _[23526]  _[28230]  _[32934]
 _[7]     _[4711]  _[9415]   _[14119]  _[18823]  _[23527]  _[28231]  _[32935]
 _[8]     _[4712]  _[9416]   _[14120]  _[18824]  _[23528]  _[28232]  _[32936]
 _[9]     _[4713]  _[9417]   _[14121]  _[18825]  _[23529]  _[28233]  _[32937]
 _[10]    _[4714]  _[9418]   _[14122]  _[18826]  _[23530]  _[28234]  _[32938]
 _[11]    _[4715]  _[9419]   _[14123]  _[18827]  _[23531]  _[28235]  _[32939]
 _[12]    _[4716]  _[9420]   _[14124]  _[18828]  _[23532]  _[28236]  _[32940]
 _[13]    _[4717]  _[9421]   _[14125

In [8]:
function set_objective_func(model, dVar, α= .8)
    @objective(model, Min, sum(α*H[i]*dVar[i,s] - (1-α)*R[i]*dVar[i,s] for i=1:n,s=1:S));
end
set_objective_func(model1, model_var, 1)

13.69 _[1] + 13.69 _[4705] + 13.69 _[9409] + 13.69 _[14113] + 13.69 _[18817] + 13.69 _[23521] + 13.69 _[28225] + 13.69 _[32929] + 10.68 _[2] + 10.68 _[4706] + 10.68 _[9410] + 10.68 _[14114] + 10.68 _[18818] + 10.68 _[23522] + 10.68 _[28226] + 10.68 _[32930] + 13.69 _[3] + 13.69 _[4707] + 13.69 _[9411] + 13.69 _[14115] + 13.69 _[18819] + 13.69 _[23523] + 13.69 _[28227] + 13.69 _[32931] + 6 _[4] + 6 _[4708] + 6 _[9412] + 6 _[14116] + 6 _[18820] + 6 _[23524] + [[...37572 terms omitted...]] + 8.4 _[14109] + 8.4 _[18813] + 8.4 _[23517] + 8.4 _[28221] + 8.4 _[32925] + 8.4 _[37629] + 8.6 _[4702] + 8.6 _[9406] + 8.6 _[14110] + 8.6 _[18814] + 8.6 _[23518] + 8.6 _[28222] + 8.6 _[32926] + 8.6 _[37630] + 8.6 _[4703] + 8.6 _[9407] + 8.6 _[14111] + 8.6 _[18815] + 8.6 _[23519] + 8.6 _[28223] + 8.6 _[32927] + 8.6 _[37631] + 8.5 _[4704] + 8.5 _[9408] + 8.5 _[14112] + 8.5 _[18816] + 8.5 _[23520] + 8.5 _[28224] + 8.5 _[32928] + 8.5 _[37632]

In [9]:
function add_general_requirements(model, dVar) 
    # Full-time; at least 36 units per semester
    @constraint(model, [s in 1:S], sum(U[i]*dVar[i, s] for i=1:n) >= 36);

    # Wellbeing: no more than 112 hours per week
    @constraint(model, [s in 1:S], sum(H[i]*dVar[i, s] for i=1:n) <= 112);

    # Not repeatable: cannot take a class again
    @constraint(model, [i in 1:n], sum(dVar[i, t] for t=1:S) <= 1);

    # Taken correct semester
    @constraint(model, [s in 1:2:S, i in 1:n], dVar[i, s] <= all_classes[i][:offered_fall]);
    @constraint(model, [s in 2:2:S, i in 1:n], dVar[i, s] <= all_classes[i][:offered_spring]);

    # 2 CI-H or HW
    @constraint(model, sum(
        (get(all_classes[i], :communication_requirement, "") in ["CI-H", "CI-HW"]) * dVar[i, s] for i=1:n, s=1:S) >= 2);
    
    # 8 HASS
    @constraint(model, sum(
        (get(all_classes[i], :hass_attribute, "") in ["HASS-H", "HASS-A", "HASS-S"]) * dVar[i, s] for i=1:n, s=1:S) >= 8);

    # 1 of each HASS-H, A, S
    @constraint(model, [hass_type in ["HASS-H", "HASS-A", "HASS-S"]], sum(
        (get(all_classes[i], :hass_attribute, "") in [hass_type]) * dVar[i, s] for i=1:n, s=1:S) >= 1);
end
add_general_requirements(model1, model_var)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["HASS-H", "HASS-A", "HASS-S"]
And data, a 3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 _[194] + _[300] + _[308] + _[309] + _[310] + _[311] + _[315] + _[329] + _[335] + _[343] + _[344] + _[346] + _[348] + _[1398] + _[1399] + _[1400] + _[1403] + _[1404] + _[1406] + _[2039] + _[2040] + _[2043] + _[2092] + _[2095] + _[2096] + _[2100] + _[2102] + _[2103] + _[2104] + _[2105] + [[...3844 terms omitted...]] + _[37526] + _[37535] + _[37538] + _[37542] + _[37543] + _[37584] + _[37585] + _[37586] + _[37587] + _[37588] + _[37591] + _[37592] + _[37593] + _[37596] + _[37599] + _[37601] + _[37604] + _[37609] + _[37612] + _[37613] + _[37614] + _[37615]

#### Adding Institute Reqs ####

In [10]:
chem_gir_ids = ["3.091", "5.111", "5.112"]
phys_1_gir_ids = ["8.01", "8.011", "8.012", "8.01L"]
phys_2_gir_ids = ["8.02", "8.021", "8.022"]
math_1_gir_ids = ["18.01", "18.01A"]
math_2_gir_ids = ["18.02", "18.02A", "18.022"]
bio_gir_ids = ["7.012", "7.013", "7.014", "7.015", "7.016"]
gir_ids = [chem_gir_ids, phys_1_gir_ids, phys_2_gir_ids, math_1_gir_ids, math_2_gir_ids, bio_gir_ids]
gir_inds_types = [[findfirst(x -> x[:subject_id] == gir_id, all_classes) for gir_id in gir_type] for gir_type in gir_ids]

6-element Vector{Vector{Int64}}:
 [2963, 3304, 3305]
 [3857, 3858, 3859, 3860]
 [3861, 3862, 3863]
 [1537, 1538]
 [1539, 1541, 1540]
 [3753, 3754, 3755, 3756, 3757]

In [11]:
function add_gir_constraints(model, dVar)
    @constraint(model, [gir_type in gir_inds_types], sum(dVar[gir_ind, s] for gir_ind in gir_type, s in 1:S) == 1);
end
add_gir_constraints(model1, model_var)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, [[2963, 3304, 3305], [3857, 3858, 3859, 3860], [3861, 3862, 3863], [1537, 1538], [1539, 1541, 1540], [3753, 3754, 3755, 3756, 3757]]
And data, a 6-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 _[2963] + _[3304] + _[3305] + _[7667] + _[8008] + _[8009] + _[12371] + _[12712] + _[12713] + _[17075] + _[17416] + _[17417] + _[21779] + _[22120] + _[22121] + _[26483] + _[26824] + _[26825] + _[31187] + _[31528] + _[31529] + _[35891] + _[36232] + _[36233] = 1
 _[3857] + _[3858] + _[3859] + _[3860] + _[8561] + _[8562] + _[8563] + _[8564] + _[13265] + _[13266] + _[13267] + _[13268] + _[17969] + _[17970] + _[17971] + _[17972] + _[22673] + _[22674] + _[22675] + _

#### Adding 6-4 Constraints ####

In [12]:
# main requirements
ai_programming_ids = ["6.100A", "6.100L"];
ai_math_1_ids = ["6.1200"];
ai_math_2_ids = ["6.S084", "18.C06", "18.06"];
ai_math_3_ids = ["6.3700", "6.3800", "18.05"];
ai_foundation_ids = ["6.1010", "6.1210"];
ai_centers_ids = ["6.1220", "6.1400", "6.3000", "6.3100", "6.3260", "6.3720", "6.3900", "6.3950",
                "6.4110", "6.4120", "6.4400", "6.4590", "6.7201", "6.C35", "9.660"] # removed "6.C571" and "6.C01"
ai_ids = [ai_programming_ids, ai_math_1_ids, ai_math_2_ids, ai_math_3_ids, ai_foundation_ids, ai_centers_ids];  # DOESN'T INCLUDE AI_MATH_IDS YET
ai_inds_types = [[findfirst(x -> x[:subject_id] == ai_id, all_classes) for ai_id in ai_type] for ai_type in ai_ids];


# additional constraints (centers)
ai_data_center_ids = ["6.3720", "6.3900"]; # no "6.C01"
ai_model_center_ids = ["6.3000", "6.3100", "6.4110", "6.4400"];
ai_dec_center_ids = ["6.3100", "6.4110", "6.7201"];  # no "6.C571"
ai_comp_center_ids = ["6.1220", "6.1400", "6.4400", "6.7201"]; # no "6.C571"
ai_human_center_ids = ["6.3260", "6.3950", "6.4120", "6.4590", "6.C35", "9.660"];
ai_centers_ids_list = [ai_data_center_ids, ai_model_center_ids, ai_dec_center_ids, ai_comp_center_ids, ai_human_center_ids];
ai_centers_inds_types = [[findfirst(x -> x[:subject_id] == ai_id, all_classes) for ai_id in ai_center] for ai_center in ai_centers_ids_list];

In [13]:
# electives
ai_app_cim = ["6.4200", "6.4210", "6.8301", "6.8611"]
ai_aus = ["18.404", "6.3730", "6.4210", "6.5151", "6.5831", "6.5931", "6.7411", "6.8301", "6.8371", "6.8611", "6.8701", "6.8711", "6.8801"] # removed "6.3020"
# 2 additional from eecs list
ai_electives_ids_list = [ai_app_cim, ai_aus]
ai_elec_inds_types = [[findfirst(x -> x[:subject_id] == ai_id, all_classes) for ai_id in ai_type] for ai_type in ai_electives_ids_list]

# other additional constraints 
ai_eecs_cim2_ids = ["6.1800", "6.1850", "6.2040", "6.2050", "6.2060", "6.2061", "6.2220", "6.2221", "6.2370", "6.2410", "6.2600", "6.4200", "6.4210", "6.4590", "6.4860", "6.4880", "6.8301", "6.8611", "6.9030", "6.UAR", "6.UAT"]
ai_serc_ids = ["6.3900", "6.3950", "6.4590", "6.8301", "6.8611"] # removed 6.C40
ai_add_constraints_ids_list = [ai_eecs_cim2_ids, ai_serc_ids]
ai_add_constraints_inds_types = [[findfirst(x -> x[:subject_id] == ai_id, all_classes) for ai_id in ai_type] for ai_type in ai_add_constraints_ids_list];

In [14]:
# ai_inds_types, 6 elem vector
function add_6_4_constraints(model, dVar)
    # 1 programming skills
    @constraint(model, sum(dVar[ai_ind, s] for ai_ind in ai_inds_types[1], s in 1:S) == 1);

    # 3 math
    ai_math_types = [ai_inds_types[2], ai_inds_types[3], ai_inds_types[4]];
    @constraint(model, [math_type in ai_math_types], sum(dVar[math_ind, s] for math_ind in math_type, s in 1:S) == 1);

    # 2 foundation
    @constraint(model, sum(dVar[ai_ind, s] for ai_ind in ai_inds_types[5], s in 1:S) == 2);

    # 5 total center
    @constraint(model, sum(dVar[ai_ind, s] for ai_ind in ai_inds_types[6], s in 1:S) >= 5);

    # 1 per center
    @constraint(model, [ai_center in ai_centers_inds_types], sum(dVar[ai_ind, s] for ai_ind in ai_center, s in 1:S) >= 1);

    # 2 from cim2
    @constraint(model, sum(dVar[ai_ind, s] for ai_ind in ai_add_constraints_inds_types[1], s in 1:S) >= 2);

    # 1 from ai+d-serc
    @constraint(model, sum(dVar[ai_ind, s] for ai_ind in ai_add_constraints_inds_types[2], s in 1:S) >= 1);

    # 1 elec from application cim and 1 from ai+d-aus
    @constraint(model, [ai_elec_type in ai_elec_inds_types], sum(dVar[ai_ind, s] for ai_ind in ai_elec_type, s in 1:S) >= 1);

    # 2 elecs ADDITIONAL from eecs or 18
    # TODO
end
add_6_4_constraints(model1, model_var)


1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, [[3454, 3455, 3589, 3601], [1623, 3442, 3455, 3488, 3511, 3517, 3563, 3589, 3594, 3601, 3607, 3609, 3614]]
And data, a 2-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 _[3454] + _[3455] + _[3589] + _[3601] + _[8158] + _[8159] + _[8293] + _[8305] + _[12862] + _[12863] + _[12997] + _[13009] + _[17566] + _[17567] + _[17701] + _[17713] + _[22270] + _[22271] + _[22405] + _[22417] + _[26974] + _[26975] + _[27109] + _[27121] + _[31678] + _[31679] + _[31813] + _[31825] + _[36382] + _[36383] + _[36517] + _[36529] ≥ 1
 _[1623] + _[3442] + _[3455] + _[3488] + _[3511] + _[3517] + _[3563] + _[3589] + _[3594] + _[3601] + _[3607] + _[3609] + _[3614] + _[63

#### Adding Prereq constraints

In [15]:
raw_prerequisites = map(course -> get(course, "prerequisites", nothing), all_classes)

4704-element Vector{Union{Nothing, String}}:
 "GIR:CAL1, ((6.100A, 6.100B)/(6.100L, 16.C20))"
 nothing
 "GIR:CAL1"
 nothing
 nothing
 nothing
 nothing
 "GIR:CAL2"
 nothing
 "''Permission of instructor''"
 "2.003, 2.016, 2.678"
 nothing
 nothing
 ⋮
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 "''One intermediate subject in French''/''permission of instructor''"
 nothing

In [16]:
abstract type ExprNode end

struct CourseNode <: ExprNode
    course::String
end

struct ANDNode <: ExprNode
    left::ExprNode
    right::ExprNode
end

struct ORNode <: ExprNode
    left::ExprNode
    right::ExprNode
end

struct PreReq
    root::CourseNode
    prereqs::Union{ExprNode, Nothing}
end 

In [17]:
function parse_prereq_string(input)
    if input === nothing
        return
    end
    input = replace(input, r"\s+" => "")
    tokens = collect(input)
    function parse(tokens::Vector{Char})::ExprNode
        stack = ExprNode[]
        buffer = IOBuffer()
        while !isempty(tokens)
            token = popfirst!(tokens)
            if token == '('
                node = parse(tokens)
                push!(stack, node)
            elseif token == ')'
                break
            elseif token == ','
                left = isempty(stack) ? CourseNode(String(take!(buffer))) : pop!(stack)
                right = parse(tokens)
                return ANDNode(left, right)
            elseif token == '/'
                left = isempty(stack) ? CourseNode(String(take!(buffer))) : pop!(stack)
                right = parse(tokens)
                return ORNode(left, right)
            else
                write(buffer, token)
            end
        end
        return isempty(stack) ? CourseNode(String(take!(buffer))) : stack[1]
    end
    return parse(tokens)
end

parse_prereq_string (generic function with 1 method)

In [18]:
function replace_gir_terms(input_str)
    if input_str === nothing
        return 
    end
    gir_map = Dict(
        "GIR:CHEM" => ["3.091", "5.111", "5.112"],
        "GIR:PHY1" => ["8.01", "8.011", "8.012", "8.01L"],
        "GIR:PHY2" => ["8.02", "8.021", "8.022"],
        "GIR:CAL1" => ["18.01", "18.01A"],
        "GIR:CAL2" => ["18.02", "18.02A", "18.022"],
        "GIR:BIOL" => ["7.012", "7.013", "7.014", "7.015", "7.016"]
    )
    pattern = r"GIR:\w+"
    replaced_str = replace(input_str, pattern => x -> begin
        term = match(pattern, x).match
        if haskey(gir_map, term)
            "(" * join(gir_map[term], "/") * ")"
        else
            term
        end
    end)
    return replaced_str
end

replace_gir_terms (generic function with 1 method)

In [19]:
prerequisites = map(prereq_str -> replace_gir_terms(prereq_str), raw_prerequisites)

4704-element Vector{Union{Nothing, String}}:
 "(18.01/18.01A), ((6.100A, 6.100B)/(6.100L, 16.C20))"
 nothing
 "(18.01/18.01A)"
 nothing
 nothing
 nothing
 nothing
 "(18.02/18.02A/18.022)"
 nothing
 "''Permission of instructor''"
 "2.003, 2.016, 2.678"
 nothing
 nothing
 ⋮
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 "''One intermediate subject in French''/''permission of instructor''"
 nothing

In [20]:
prerequisite_expressions = []
for index in 1:length(prerequisites)
    course_id = CourseNode(index_to_subjectid[index])
    prereq_tree = parse_prereq_string(prerequisites[index])
    push!(prerequisite_expressions, PreReq(course_id, prereq_tree))
end
prerequisite_expressions

4704-element Vector{Any}:
 PreReq(CourseNode("1.00"), ANDNode(ORNode(CourseNode("18.01"), CourseNode("18.01A")), ORNode(ANDNode(CourseNode("6.100A"), CourseNode("6.100B")), ANDNode(CourseNode("6.100L"), CourseNode("16.C20")))))
 PreReq(CourseNode("1.000"), nothing)
 PreReq(CourseNode("1.001"), ORNode(CourseNode("18.01"), CourseNode("18.01A")))
 PreReq(CourseNode("1.005"), nothing)
 PreReq(CourseNode("1.007"), nothing)
 PreReq(CourseNode("1.008"), nothing)
 PreReq(CourseNode("1.009"), nothing)
 PreReq(CourseNode("1.010"), ORNode(CourseNode("18.02"), ORNode(CourseNode("18.02A"), CourseNode("18.022"))))
 PreReq(CourseNode("1.011"), nothing)
 PreReq(CourseNode("1.013"), CourseNode("''Permissionofinstructor''"))
 PreReq(CourseNode("1.015"), ANDNode(CourseNode("2.003"), ANDNode(CourseNode("2.016"), CourseNode("2.678"))))
 PreReq(CourseNode("1.016"), nothing)
 PreReq(CourseNode("1.018"), nothing)
 ⋮
 PreReq(CourseNode("WGS.250"), nothing)
 PreReq(CourseNode("WGS.270"), nothing)
 PreReq(Course

In [23]:
function add_course_constraint(model::Model, dVar, course::CourseNode, sem::Int)
    index = get(subjectid_to_index, course.course, nothing)
    if index == nothing
        return 0  # Ignore unknown courses
    end
    if sem == 1
        return 0  # Cannot satisfy prerequisites in the first semester
    else
        return handle_prerequisite_sum(model, dVar, index, sem)
    end
end

function handle_prerequisite_sum(model::Model, dVar, index::Int, sem::Int)
    prereq_sum = sum(dVar[index, s] for s in 1:sem-1)
    binary_prereq_var = @variable(model, binary=true)
    @constraint(model, binary_prereq_var <= prereq_sum)
    @constraint(model, binary_prereq_var >=  prereq_sum / (sem-1))  # Ensure binary behavior
    return binary_prereq_var
end

function add_and_constraint(left_var, right_var, model::Model)
    and_var = @variable(model, binary=true)
    @constraint(model, and_var <= left_var)
    @constraint(model, and_var <= right_var)
    @constraint(model, and_var >= left_var + right_var - 1)
    return and_var
end

function add_or_constraint(left_var, right_var, model::Model)
    or_var = @variable(model, binary=true)
    @constraint(model, or_var >= left_var)
    @constraint(model, or_var >= right_var)
    @constraint(model, or_var <= left_var + right_var)
    return or_var
end

function add_and_or_constraints(model::Model, dVar, expr::ExprNode, sem::Int)
    if isa(expr, CourseNode)
        return add_course_constraint(model, dVar, expr, sem)
    elseif isa(expr, ANDNode)
        left_var = add_and_or_constraints(model, dVar, expr.left, sem)
        right_var = add_and_or_constraints(model, dVar, expr.right, sem)
        return add_and_constraint(left_var, right_var, model)
    elseif isa(expr, ORNode)
        left_var = add_and_or_constraints(model, dVar, expr.left, sem)
        right_var = add_and_or_constraints(model, dVar, expr.right, sem)
        return add_or_constraint(left_var, right_var, model)
    else
        error("Unknown expression type")
    end
end

function add_prereq_constraint(model, dVar, prereq_obj, sem)
    index = get(subjectid_to_index, prereq_obj.root.course, nothing)
    if index == nothing
        return 1  # Ignore unknown courses
    end
    if prereq_obj.prereqs === nothing
        return dVar[index, sem]
    else
        prereq_var = add_and_or_constraints(model, dVar, prereq_obj.prereqs, sem)
        @constraint(model, dVar[index, sem] <= prereq_var)
        return dVar[index, sem]
    end
end


add_prereq_constraint (generic function with 1 method)

In [24]:
function add_prereq_constraints(model, dVar) 
    for s in 1:S
        for prereq_expr in prerequisite_expressions
            add_prereq_constraint(model, dVar, prereq_expr, s)
        end
    end
end 

add_prereq_constraints(model1, model_var)

#### Optimizing the Model and Interpretation

In [25]:
optimize!(model1)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 21.5.0 21F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 211874 rows, 96680 columns and 681727 nonzeros
Model fingerprint: 0xa094f64d
Variable types: 0 continuous, 96680 integer (96680 binary)
Coefficient statistics:
  Matrix range     [1e-01, 8e+01]
  Objective range  [1e+00, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 180914 rows and 76407 columns
Presolve time: 0.96s
Presolved: 30960 rows, 20273 columns, 109311 nonzeros
Variable types: 0 continuous, 20273 integer (20273 binary)
Found heuristic solution: objective 639.7100000
Found heuristic solution: objective 392.3800000

Root relaxation: objective 2.167867e+02, 2549 iterations, 0.07 seconds (0.15 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   G

In [26]:
function interpret_course_nums(classes_id_per_sem)
    course_nums = [[all_classes[class_ind][:subject_id] for class_ind in sem] for sem in classes_id_per_sem]
    return course_nums
end

function interpret_course_attr(classes_id_per_sem, attr)
    courses_attr = [[get(all_classes[class_ind], Symbol(attr), "") for class_ind in sem] for sem in classes_id_per_sem]
    return courses_attr
end

function interpret_course_hours(classes_id_per_sem)
    course_hours = [[H[class_ind] for class_ind in sem] for sem in classes_id_per_sem]
    return course_hours
end

function interpret_course_units(classes_id_per_sem)
    course_units = [[U[class_ind] for class_ind in sem] for sem in classes_id_per_sem]
    return course_units
end

interpret_course_units (generic function with 1 method)

In [29]:
taken_classes_sem = [[ind for (ind, val) in enumerate(sem) if val == 1] for sem in eachcol(value.(model_var))]
interpret_course_nums(taken_classes_sem)

8-element Vector{Vector{String}}:
 ["21M.747", "21W.032", "8.01L"]
 ["18.01", "21M.460", "6.100A", "ES.113"]
 ["18.022", "21M.451", "21M.623", "6.1200"]
 ["6.1010", "6.1210", "8.02"]
 ["18.06", "21M.622", "6.3700"]
 ["6.3900", "6.8301", "6.UAT"]
 ["6.3100", "6.3950", "7.012"]
 ["3.091", "6.1400", "6.3260"]

In [30]:
interpret_course_attr(taken_classes_sem, "title")

8-element Vector{Vector{String}}:
 ["Talking and Dancing", "Science Writing and New Media: Introduction to Digital Media", "Physics I"]
 ["Calculus", "MIT Senegalese Drum Ensemble", "Introduction to Computer Science Programming in Python", "Ancient Greek Philosophy and Mathematics"]
 ["Calculus", "Collaborative Piano", "Physical Improvisation: Bodies in Motion", "Mathematics for Computer Science"]
 ["Fundamentals of Programming", "Introduction to Algorithms", "Physics II"]
 ["Linear Algebra", "Physical Improvisation: Scores and Structures", "Introduction to Probability"]
 ["Introduction to Machine Learning", "Advances in Computer Vision", "Oral Communication"]
 ["Dynamical System Modeling and Control Design", "AI, Decision Making, and Society", "Introductory Biology"]
 ["Introduction to Solid-State Chemistry", "Computability and Complexity Theory", "Networks"]

In [ ]:
prerequisite_expressions[subjectid_to_index["16.84"]]

In [ ]:
sems_hours = interpret_course_hours(taken_classes_sem)
[sum(sem_hours) for sem_hours in sems_hours]

In [33]:
function get_course_path(major, alpha=.8)
    model = create_model(GRB_ENV)
    course_var = add_course_semester_variables!(model)
    set_objective_func(model, course_var, alpha)
    add_general_requirements(model, course_var)
    add_6_4_constraints(model, course_var) # change for major
    add_prereq_constraints(model, course_var)
    optimize!(model)
    taken_classes_sem = [[ind for (ind, val) in enumerate(sem) if val == 1] for sem in eachcol(value.(course_var))]
    return interpret_course_nums(taken_classes_sem)
end

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 21.5.0 21F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 211868 rows, 96680 columns and 681567 nonzeros
Model fingerprint: 0x30921b65
Variable types: 0 continuous, 96680 integer (96680 binary)
Coefficient statistics:
  Matrix range     [1e-01, 8e+01]
  Objective range  [2e-02, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 180171 rows and 76164 columns
Presolve time: 0.96s
Presolved: 31697 rows, 20516 columns, 110844 nonzeros
Variable types: 0 continuous, 20516 integer (20516 binary)
Found heuristic solution: objective 412.0100000
Found heuristic solution: objective 240.4900000
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.01s

Solved with dual simplex

Root relaxation: objective 1.200192e+02, 1785 iterations, 0.11 seconds (0.14 

8-element Vector{Vector{String}}:
 ["18.01", "21M.747", "6.3950"]
 ["1.267", "15.3941", "6.100A", "6.UAT"]
 ["18.022", "21M.623", "6.1010"]
 ["18.06", "6.1200", "STS.083"]
 ["21W.032", "6.1210", "6.3700"]
 ["6.3260", "6.4110", "ES.113"]
 ["10.960", "21M.460", "21M.622", "24.93", "3.903", "6.3900"]
 ["15.323", "6.1400", "6.8301", "HST.533"]

# Web Stuff

In [ ]:
Pkg.add("Genie")

In [ ]:
using Genie, Genie.Renderer.Html, Genie.Requests

form = """
<form action="/" method="POST" enctype="multipart/form-data">
  <input type="text" name="name" value="" placeholder="What's your name?" />
  <input type="submit" value="Greet" />
</form>
"""

route("/") do
  html(form)
end

route("/", method = POST) do
  "Hello $(postpayload(:name, "Anon"))"
end

up()